In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import h5py

In [122]:
# Functions
# def get_crop_mask(margin_px=30, ):

fdn_Yike = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/figures/'
fdn_new = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/cancer100cells/'
mask_fdn = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/whole_figure_mask/'

obs = pd.read_csv('/home/yike/phd/cancer_cells_img_seq/data/20220201_NextSeq/raw_with_gene_names_obs.tsv', sep='\t', index_col=0)

orig_fns = [i for i in os.listdir(fdn_new) if ('.' in i) & (i.split('.')[0] + '.tiff' not in os.listdir(mask_fdn))]
ncells = len(orig_fns)
for i, fn in enumerate(orig_fns):
    cellname = fn.split('.')[0]
    print(f'{cellname}: {i+1}/{ncells}')

    grid = obs.loc[cellname]['grid']
    
    print('load...')
    raw = np.load(fdn_Yike+fn)
    seg = raw['segmentation']

    print('find ROI...')
    i0s = seg.any(axis=1).nonzero()[0]
    i1s = seg.any(axis=0).nonzero()[0]
    i00, i01 = i0s[0], i0s[-1] + 1
    i10, i11 = i1s[0], i1s[-1] + 1

    print('add margins...')
    
    if 'G7_S57' in fn:
        margin_px = 60
    else:
        margin_px = 30
    i00 = max(0, i00 - margin_px)
    i10 = max(0, i10 - margin_px)
    i01 = min(seg.shape[0], i01 + margin_px)
    i11 = min(seg.shape[1], i11 + margin_px)

    mask = np.zeros([4096, 4096])
    crop_fn = fdn_new + fn
    mask[i00: i01, i10: i11] = 1 * np.load(fdn_new + fn)['new_segmentation']

    mask_mat = Image.fromarray(mask.T)
    mask_mat.save(mask_fdn + grid + '-' + cellname + '.tiff')

C6_S40: 1/96
load...
find ROI...
add margins...
I8_S72: 2/96
load...
find ROI...
add margins...
J3_S8: 3/96
load...
find ROI...
add margins...
H10_S97: 4/96
load...
find ROI...
add margins...
G11_S109: 5/96
load...
find ROI...
add margins...
C7_S53: 6/96
load...
find ROI...
add margins...
L3_S10: 7/96
load...
find ROI...
add margins...
C5_S27: 8/96
load...
find ROI...
add margins...
E11_S107: 9/96
load...
find ROI...
add margins...
J12_S125: 10/96
load...
find ROI...
add margins...
D5_S28: 11/96
load...
find ROI...
add margins...
H8_S71: 12/96
load...
find ROI...
add margins...
F8_S69: 13/96
load...
find ROI...
add margins...
G9_S83: 14/96
load...
find ROI...
add margins...
M4_S24: 15/96
load...
find ROI...
add margins...
G5_S31: 16/96
load...
find ROI...
add margins...
J8_S73: 17/96
load...
find ROI...
add margins...
I10_S98: 18/96
load...
find ROI...
add margins...
I7_S59: 19/96
load...
find ROI...
add margins...
F12_S121: 20/96
load...
find ROI...
add margins...
G7_S57: 21/96
load..

In [24]:
import subprocess as sp
path = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/whole_figure_mask/'
obs = pd.read_csv('/home/yike/phd/cancer_cells_img_seq/data/20220201_NextSeq/raw_with_gene_names_obs.tsv', 
                  sep='\t', index_col=0)
files = os.listdir(path)
for file in files:
    name = file.split('-')[1].split('.')[0]
    mask_name = obs.loc[name]['mask_name'] + '.tiff'
    command = 'mv ' + path + file + ' ' + path + mask_name
    sp.run(command, shell=True)